# 1/ load csv realestate_fraugster_case.csv
# 2/ clean the data best you think 
# 3/ write a new csv with a similar name with the cleaned data 
# 4/ output the following answers: 
	a. what is the distance (in meters) between the cheapest sale and the most recent sale?
	b. what is the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998 , in Sacramento?
	c. what is the city name, and its 3 most common zip codes, that has the 2nd highest amount of beds sold? 



# Note 

### Size of the dataset:
(986, 12)
### Missing value: 
We have some missing values, some data are equal to zero we will have to know if it's a real missing data or a real 0. We will not take the risk to remove the missing values or replace due to the size of the dataset 
we will harmonize the missing data with NaN.
### types
A lot of columuns are not in a good format, before to convert it we will have to clean our columns




In [304]:
import pandas as pd 
import numpy as np
import seaborn as sns
import string
pd.set_option("max_rows", None)

In [305]:

punctuation = '!"#$%&\'()*+,./;<=>?@[\\]^_`{|}%~'
#alphabet list
a=ord('a')
alph=[chr(i) for i in range(a,a+26)]
#Upper alphabet
alph_upper=[x.upper() for x in alph]
alphabet=alph+alph_upper

#state,type, cuty
def pipeline_cleaning(test):
    test=str(test)
    test=test.upper()
    test_punct_removed=[char for char in test if char not in string.punctuation]
    test_punct_removed_join=''.join(test_punct_removed)
    result = ''.join([i for i in test_punct_removed_join if not i.isdigit()])
    return result


def clean_date(x):
    test_alphabet_removed=[char for char in x if char not in alphabet]
    test_alphabet_removed_join=''.join(test_alphabet_removed)
    test_punc_removed=[char for char in test_alphabet_removed_join if char not in punctuation]
    test_punc_removed_join=''.join(test_punc_removed)
    result=pd.to_datetime(test_punc_removed_join, format='%Y/%m/%d')
    return result


def clean_zip(x):
    x=str(x)
    result = ''.join([i for i in x if i.isdigit()])
    return result



    

# 1. load csv realestate_fraugster_case.csv

In [306]:
data=pd.read_csv('realestate_fraugster_case.csv', sep=';')
df=data.copy()

In [327]:
df.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2.0,1.0,836.0,RESIDENTIAL,1943-01-09 11:56:01,59222.0,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3.0,1.0,1167.0,RESIDENTIAL,1996-11-08 23:09:38,68212.0,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2.0,1.0,796.0,RESIDENTIAL,1915-01-05 07:31:45,68880.0,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2.0,1.0,852.0,RESIDENTIAL,1998-10-22 04:46:05,69307.0,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2.0,1.0,797.0,RESIDENTIAL,1972-01-05 20:52:32,81900.0,38.51947,-121.435768


In [308]:
df.shape

(986, 12)

Very small Dataset 986 lines, a lot of non numeric elements are in the numeric columnand numeric element in non numeric column 

In [309]:
df.dtypes

street        object
city          object
zip           object
state         object
beds         float64
baths        float64
sq__ft       float64
type          object
sale_date     object
price         object
latitude      object
longitude     object
dtype: object

### observation
A lot of numerics elements are not in the good format like price, lattitude, longitude, sale_date etc

# 2. clean the data best you think

### state, type, city
We need to create a pipeline to:
<li>transform the column in a string format</li>
<li>Use only upper case</li>
<li>remove the punctuation</li>
<li>remove the digit</li>

In [310]:
df['state']=df['state'].apply(pipeline_cleaning)
df['type']=df['type'].apply(pipeline_cleaning)
df['city']=df['city'].apply(pipeline_cleaning)

### zip code 

In [311]:
df['zip']=df['zip'].apply(clean_zip)

### sale date 
we need to create a function to:
<li>clean the column (string, symbol etc)</li>
<li>Put the column in a time format</li>

In [312]:
df['sale_date']=df['sale_date'].apply(clean_date)

### Price
We keep only the digits in a type float

In [313]:

df['price'] = (df['price'].str.replace(r'\D', '')).astype(float)

### Baths
We have some elements with 0 bedroom it could be a studio so we will not replace it, for the bathroom the problem is different we can see that the way the 0 are recorded seems suspsicious we have a series of zero for a specific district, we cannot contact the data provider we will just replace by np.nan for the moment.

In [314]:
df['baths'].replace(0, np.nan, inplace=True)

A sq__ft of 0 is something impossible, we need to replace le 0 value by nan to specify we don't know the value

### Square feet

In [315]:
df['sq__ft'].replace(0, np.nan, inplace=True)

For the street element we can see that few streets are recorded as E st or 1st street after looking on internet it seems that this 2 streets exists.

### Missing value
We will remove or replace any missing value, with the last steps we harmonized the missing value.


In [316]:
df.isna().sum()/df.shape[0]

street       0.001014
city         0.000000
zip          0.000000
state        0.000000
beds         0.001014
baths        0.110548
sq__ft       0.174442
type         0.000000
sale_date    0.000000
price        0.001014
latitude     0.001014
longitude    0.001014
dtype: float64

### Strategy against missing value
Before to use a strategy for missing value like apply an imputer from sklearn, fillna() or dropna() I need to get more information about the purpose of the study, I prefer to keep the dataset with NaN until to get new information

# 3/ write a new csv with a similar name with the cleaned data

In [317]:
df.to_csv(r'/Users/robin/Desktop/fraugster/clean_df.csv', index = False)
print('save')

save


# 4/ output the following answers:

## a. what is the distance (in meters) between the cheapest sale and the most recent sale?


In [318]:
cheapest_price=df['price'].min()
cheapest_price

1551.0

In [319]:
recent_date = df['sale_date'].max()
recent_date

Timestamp('2019-11-01 23:37:06')

In [320]:
cheapest=df.loc[(df["price"]==cheapest_price)]
recent=df.loc[(df["sale_date"]==recent_date)]
print(cheapest, recent)

                   street     city    zip state  beds  baths  sq__ft  \
865  3720 VISTA DE MADERA  LINCOLN  95648    CA   3.0    3.0     NaN   

            type           sale_date   price   latitude    longitude  
865  RESIDENTIAL 1925-12-05 14:57:47  1551.0  38.851645  -121.231742                    street     city    zip state  beds  baths  sq__ft  \
561  48 CRYSTALWOOD CIR  LINCOLN  95648    CA   0.0    NaN     NaN   

            type           sale_date   price   latitude    longitude  
561  RESIDENTIAL 2019-11-01 23:37:06  4897.0  38.884667  -121.289896  


In [321]:
import geopy.distance

coords_1 = (38.851645, -121.231742)
coords_2 = (38.884667, 121.289896)

print('The distance between the cheapest sale and the most recent sale is',geopy.distance.distance(coords_1, coords_2).m,'meters')



The distance between the cheapest sale and the most recent sale is 9301048.898407102 meters


## b. what is the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998 , in Sacramento?


In [322]:
dx_b=df[(df['type']=='MULTIFAMILY') & (df["city"]=='SACRAMENTO')&(df["sale_date"] > '1933-05-11') & (df["sale_date"] < '1998-03-12')]
dx_b

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
113,10158 CRAWFORD WAY,SACRAMENTO,95827,CA,4.0,4.0,2213.0,MULTIFAMILY,1953-04-21 02:32:12,297000.0,38.5703,-121.315735
353,2820 DEL PASO BLVD,SACRAMENTO,95815,CA,4.0,2.0,1404.0,MULTIFAMILY,1944-01-09 17:10:33,100000.0,38.617718,-121.440089
527,398 LINDLEY DR,SACRAMENTO,95815,CA,4.0,2.0,1744.0,MULTIFAMILY,1984-11-07 08:49:22,416767.0,38.622359,-121.457582


In [323]:
dx_b['number']=dx_b.street.str.split(' ').str[0]
dx_b['number']=dx_b['number'].astype('int32')
median_number=dx_b['number'].median()
print('the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998 , in Sacramento is:',median_number)

the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998 , in Sacramento is: 2820.0


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## c. what is the city name, and its 3 most common zip codes, that has the 2nd highest amount of beds sold

In [324]:
dx_group=df[['city','beds']].groupby(["city"])
dx_group_sum =dx_group.sum()
dx_group_sum_sort=dx_group_sum.sort_values(by=['beds'],ascending=False)
dx_group_sum_sort.head()

,beds
city,
SACRAMENTO,1329.0
ELK GROVE,383.0
ROSEVILLE,133.0
CITRUS HEIGHTS,113.0
ANTELOPE,111.0


ELK GROVE is the second city with the highest amount of bed sold

### By most common zip codes I interpret as the one with most importants number of sales  

In [328]:
dx_ELK_GROVE=df.loc[(df["city"]=='ELK GROVE')]
dx_ELK_GROVE.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
30,5201 LAGUNA OAKS DR Unit 140,ELK GROVE,95758,CA,2.0,2.0,1039.0,CONDO,1995-06-27 11:07:11,133000.0,38.423251,-121.444489
34,5201 LAGUNA OAKS DR Unit 162,ELK GROVE,95758,CA,2.0,2.0,1039.0,CONDO,1995-02-14 02:55:54,141000.0,38.423251,-121.444489
42,8718 ELK WAY,ELK GROVE,95624,CA,3.0,2.0,1056.0,RESIDENTIAL,1964-12-06 07:32:56,156896.0,38.41653,-121.379653
50,9417 SARA ST,ELK GROVE,95624,CA,3.0,2.0,1188.0,RESIDENTIAL,1913-11-26 14:12:17,170000.0,38.415518,-121.370527
66,7005 TIANT % WAY,ELK GROVE,95758,CA,3.0,2.0,1586.0,RESIDENTIAL,1911-11-05 06:37:27,194000.0,38.422811,-121.423285


The city name, and its 3 most common zip codes, that has the 2nd highest amount of beds sold are:

In [326]:
dx_ELK_GROVE['zip'].value_counts()

95758    44
95757    36
95624    34
Name: zip, dtype: int64